In [1]:
!pip install timm scipy scikit-learn torchcam --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 49.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is in

In [1]:
import os
import torch
import timm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile

ZIP_PATH = "/content/drive/MyDrive/cars196/dataset/archive (7).zip"
BASE_DIR = "/content/cars196"

os.makedirs(BASE_DIR, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

print("Extracted:", os.listdir(BASE_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted: ['car_devkit', 'cars_test', 'cars_train']


In [5]:
TRAIN_IMG_DIR = "/content/cars196/cars_train/cars_train"
TEST_IMG_DIR  = "/content/cars196/cars_test/cars_test"

DEVKIT = "/content/cars196/car_devkit/devkit"
TRAIN_ANNO = DEVKIT + "/cars_train_annos.mat"
TEST_ANNO  = DEVKIT + "/cars_test_annos.mat"


In [19]:
class CarsDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["image"]).convert("RGB")
        x1, y1, x2, y2 = row["bbox"]
        img = img.crop((x1, y1, x2, y2))
        label = row["label"]
        if self.transform:
            img = self.transform(img)
        return img, label

train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_ds = CarsDataset(train_df, train_tf)
val_ds   = CarsDataset(val_df, val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
model = timm.create_model("convnext_tiny", pretrained=True, num_classes=196)
model = model.to(device)

# unfreezing entire model
for param in model.parameters():
    param.requires_grad = True

# separate LR for head and backbone 
head_params = list(model.head.parameters())
backbone_params = [p for n,p in model.named_parameters() if "head" not in n]

optimizer = torch.optim.AdamW([
    {'params': backbone_params, 'lr': 1e-4},
    {'params': head_params, 'lr': 3e-4}
], weight_decay=1e-4)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


In [21]:
def train_epoch(model, loader):
    model.train()
    total, correct, loss_sum = 0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        correct += (preds.argmax(1) == y).sum().item()
        total += y.size(0)
    return loss_sum/len(loader), correct/total

def eval_epoch(model, loader):
    model.eval()
    total, correct, loss_sum = 0, 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            loss = criterion(preds, y)
            loss_sum += loss.item()
            correct += (preds.argmax(1) == y).sum().item()
            total += y.size(0)
    return loss_sum/len(loader), correct/total


In [22]:
EPOCHS = 20
best_val = 0
SAVE_DIR = "/content/drive/MyDrive/terafac_models"
os.makedirs(SAVE_DIR, exist_ok=True)

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss, val_acc = eval_epoch(model, val_loader)
    scheduler.step()

    if val_acc > best_val:
        best_val = val_acc
        torch.save(model.state_dict(), f"{SAVE_DIR}/convnext_tiny_best.pth")

    print(f"Epoch {epoch+1:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

print("Best Val Acc:", best_val)


Epoch 01 | Train Acc: 0.2250 | Val Acc: 0.6728
Epoch 02 | Train Acc: 0.8379 | Val Acc: 0.8220
Epoch 03 | Train Acc: 0.9613 | Val Acc: 0.8877
Epoch 04 | Train Acc: 0.9880 | Val Acc: 0.9091
Epoch 05 | Train Acc: 0.9942 | Val Acc: 0.9147
Epoch 06 | Train Acc: 0.9952 | Val Acc: 0.9184
Epoch 07 | Train Acc: 0.9966 | Val Acc: 0.9147
Epoch 08 | Train Acc: 0.9969 | Val Acc: 0.9214
Epoch 09 | Train Acc: 0.9975 | Val Acc: 0.9220
Epoch 10 | Train Acc: 0.9982 | Val Acc: 0.9233
Epoch 11 | Train Acc: 0.9977 | Val Acc: 0.9263
Epoch 12 | Train Acc: 0.9980 | Val Acc: 0.9227
Epoch 13 | Train Acc: 0.9986 | Val Acc: 0.9288
Epoch 14 | Train Acc: 0.9986 | Val Acc: 0.9276
Epoch 15 | Train Acc: 0.9985 | Val Acc: 0.9288
Epoch 16 | Train Acc: 0.9982 | Val Acc: 0.9294
Epoch 17 | Train Acc: 0.9986 | Val Acc: 0.9257
Epoch 18 | Train Acc: 0.9992 | Val Acc: 0.9269
Epoch 19 | Train Acc: 0.9988 | Val Acc: 0.9276
Epoch 20 | Train Acc: 0.9992 | Val Acc: 0.9269
Best Val Acc: 0.9294045426642111


In [23]:
# Save the best ConvNeXt-Tiny weights
torch.save(model.state_dict(), "/content/drive/MyDrive/terafac_models/convnext_tiny_best.pth")


In [24]:
# Save the final trained model (optional, but good to have)
torch.save(model.state_dict(), "/content/drive/MyDrive/terafac_models/convnext_tiny_final.pth")
